# Loading Team State from a File

hm state ko file mein save krenge ki agar kbhi crash ya fir koi issue aata hai to us file se state ko load kr skenge

In [1]:
import asyncio
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage

model_client = OllamaChatCompletionClient(model="gemma2:latest")

In [2]:
import json

with open('team_state.json','r') as f:
    team_state = json.load(f)

team_state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'writer1': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Write a poem about the sea in 3 lines',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': 'Salt air whispers secrets untold,\nWaves crash and sigh, an endless gold.\nThe ocean breathes, both fierce and bold. \n\n\n',
       'thought': None,
       'source': 'writer1',
       'type': 'AssistantMessage'}]}},
   'message_buffer': [{'id': 'a5b1b9e1-9790-41d8-a334-a8cf3252ded0',
     'source': 'writer2',
     'models_usage': {'prompt_tokens': 65, 'completion_tokens': 30},
     'metadata': {},
     'created_at': '2025-10-10T10:35:04.143390Z',
     'content': "The boundless blue, a restless soul,\nWhere silver foam on sunbeams stroll,\nAnd crashing waves reach heaven's goal.  \n",
     'type': 'TextMessage'}]},
  'writer2'

In [3]:
from autogen_agentchat.conditions import MaxMessageTermination
agent1 = AssistantAgent(
    name = "writer1",
    model_client=model_client,
    system_message="You are a contemporary writer. Give the output in less than 30 words."
)

agent2 = AssistantAgent(
    name = "writer2",
    model_client=model_client,
    system_message="You are a classic writer. Give the output in less than 30 words."
)

termination_condition = MaxMessageTermination(max_messages=3)

new_agent_team = RoundRobinGroupChat(participants=[agent1,agent2], termination_condition=termination_condition)

In [6]:
await new_agent_team.load_state(team_state)

In [7]:
from autogen_agentchat.ui import Console
stream = new_agent_team.run_stream(task="what is the last line of poem you wrote")
await Console(stream)

---------- TextMessage (user) ----------
what is the last line of poem you wrote
---------- TextMessage (writer1) ----------
The last line of the poem is: "And crashing waves reach heaven's goal." 

---------- TextMessage (writer2) ----------
You are quite right! I seem to have gotten lost in the rhythm of the sea myself for a moment. 😊  



TaskResult(messages=[TextMessage(id='90e811e2-2660-4960-bc3c-93da435bd819', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 40, 25, 912527, tzinfo=datetime.timezone.utc), content='what is the last line of poem you wrote', type='TextMessage'), TextMessage(id='b5017b70-ae6d-4e3d-90b6-d8071c2c950c', source='writer1', models_usage=RequestUsage(prompt_tokens=112, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 40, 40, 476776, tzinfo=datetime.timezone.utc), content='The last line of the poem is: "And crashing waves reach heaven\'s goal." \n', type='TextMessage'), TextMessage(id='ac715c50-5d67-498b-ab49-213141ae7545', source='writer2', models_usage=RequestUsage(prompt_tokens=134, completion_tokens=26), metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 40, 58, 82544, tzinfo=datetime.timezone.utc), content='You are quite right! I seem to have gotten lost in the rhythm of the sea myself for a moment. 😊  \n